In [4]:
!git config --global user.name "rachitneedcse"
!git config --global user.email "rachitguptacse.098@gmail.com"

# dependecy

In [1]:
pip install sentence-transformers langchain_community unstructured faiss-cpu Together together


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 50.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.7/372.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 85.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/1

# Code

In [2]:
import json


with open("/kaggle/input/final-database/legal_database.json", "r", encoding="utf-8") as f:
    qa_data = json.load(f)


questions = [item["question"] for item in qa_data]
answers = {item["question"]: item["answer"] for item in qa_data}  # Mapping Q -> A


In [3]:
from sentence_transformers import SentenceTransformer


model = SentenceTransformer("sentence-transformers/msmarco-distilbert-base-v4")  
model.save("qa_model")

question_embeddings = model.encode(questions, convert_to_tensor=True)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/455 [00:00<?, ?it/s]

In [4]:
import torch


torch.save({
    "embeddings": question_embeddings,
    "questions": questions
}, "question_embeddings.pt")


In [5]:
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFDirectoryLoader


txt_loader = DirectoryLoader('/kaggle/input/indian-law', glob="*.txt")
pdf_loader = PyPDFDirectoryLoader('/kaggle/input/indian-law')  

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)


txt_documents = txt_loader.load_and_split(text_splitter)
pdf_documents = pdf_loader.load_and_split(text_splitter)


all_documents = txt_documents + pdf_documents
chunks = [doc.page_content for doc in all_documents]


np.save("legal_chunks.npy", np.array(chunks, dtype=object))

print(f"✅ Processed {len(chunks)} legal text chunks from TXT & PDF.")


✅ Processed 20628 legal text chunks from TXT & PDF.


In [6]:
chunks[1]

'[Received the assent of the Governor-General on October 6, 1860.]\n\nCHAPTER I INTRODUCTION\n\nThe Indian Penal Code was drafted by the First Indian Law Commission presided over by Lord Thomas Babington Macaulay. The draft underwent further revision at the hands of well-known jurists, like Sir Barnes Peacock, and was completed in 1850. The Indian Penal Code was passed by the then Legislature on 6 October 1860 and was enacted as Act No. XLV of 1860.'

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings



embedding_model = HuggingFaceEmbeddings(model_name="law-ai/InLegalBERT")


legal_embeddings = np.array([embedding_model.embed_query(chunk) for chunk in chunks])


np.save("legal_embeddings.npy", legal_embeddings)


<ipython-input-7-d78e25d61bc3>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="law-ai/InLegalBERT")


config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/534M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
model_path = "legal_model"
embedding_model.client.save_pretrained(model_path) 

In [9]:
import faiss

legal_embeddings = np.load("/kaggle/working/legal_embeddings.npy", allow_pickle=True)
dim = legal_embeddings.shape[1]


faiss_index = faiss.IndexFlatL2(dim)


faiss_index.add(legal_embeddings)


faiss.write_index(faiss_index, "legal_faiss.index")


In [10]:
import together
together.api_key = "tgp_v1_hcAYeb5IquESKVQOsx6_wbAn0jkRNJTWHnNipFUTIlI"

In [11]:
import json
import torch
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, util
from langchain_community.embeddings import HuggingFaceEmbeddings


model = SentenceTransformer("/kaggle/working/qa_model")
embedding_model = HuggingFaceEmbeddings(model_name="/kaggle/working/legal_model")


data = torch.load("question_embeddings.pt",weights_only=False)
question_embeddings = data["embeddings"]
questions = data["questions"]
faiss_index = faiss.read_index("/kaggle/working/legal_faiss.index")
legal_chunks = np.load("/kaggle/working/legal_chunks.npy", allow_pickle=True)



def find_closest_match(user_question):
    
    user_embedding = model.encode(user_question, convert_to_tensor=True)
    
    similarities = util.pytorch_cos_sim(user_embedding, question_embeddings)
    best_match_idx = similarities.argmax().item()
    
    best_match_question = questions[best_match_idx]
    similarity_score = similarities[0][best_match_idx].item()

    return best_match_question, similarity_score


def search_legal_docs(query,top_k=5):
    
    query_embedding = np.array(embedding_model.embed_query(query)).reshape(1, -1)
    distances, indices = faiss_index.search(query_embedding, top_k)
    
    retrieved_chunks = [legal_chunks[idx] for idx in indices[0]]

    
    query_vector = torch.tensor(query_embedding, dtype=torch.float32)
    chunk_vectors = torch.tensor([embedding_model.embed_query(chunk) for chunk in retrieved_chunks], dtype=torch.float32)

    scores = util.pytorch_cos_sim(query_vector, chunk_vectors)[0]
    ranked_chunks = sorted(zip(retrieved_chunks, scores), key=lambda x: x[1], reverse=True)
    best_chunk, _ = ranked_chunks[0]
    model_name = "mistralai/Mistral-7B-Instruct-v0.2"  # Choose another if needed

    prompt = f"""
    As a legal chatbot specializing in the Indian Penal Code, you are tasked with providing highly accurate and contextually appropriate responses. Ensure your answers meet these criteria:
- First of all state the law in which the context comes in. Then Respond in a 5 bullet-point format to clearly delineate distinct aspects of the legal query.
- Each point should accurately reflect the legal provision in question, avoiding over-specificity unless directly relevant to the user's query.
- Clarify the general applicability of the legal rules or sections mentioned, highlighting any common misconceptions or frequently misunderstood aspects.
- Limit responses to essential information that directly addresses the user's question, providing concise yet comprehensive explanations.
- Avoid assuming specific contexts or details not provided in the query, focusing on delivering universally applicable legal interpretations unless otherwise specified.


CONTEXT: {best_chunk}

QUESTION: {query}
ANSWER:


    """
    
    response = together.Complete.create(
        model=model_name,
        prompt=prompt,
        max_tokens=400,
        temperature=1
    )
    

    if "choices" in response and response["choices"]:
        
        return response["choices"][0]["text"].strip()
    else:
        return "Error: No response received from Together AI."

def get_answer(user_question):
    
    best_match_question, similarity_score = find_closest_match(user_question)

    if similarity_score >= 0.7:
        return f"✅ Exact Match Found({similarity_score:.2f}):\n{answers[best_match_question]}"

    elif 0.3 < similarity_score < 0.7:
        top_chunks = search_legal_docs(user_question)
        return f"⚖️ Legal Text Found:\n{top_chunks}"  

    else:
        return "Sorry cant give response at the moment."  


user_question = "i got in an accident. What should i do?"
response = get_answer(user_question)
print(response)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-11-bc25cf7ecf92>:67: DeprecationWarning: Call to deprecated function create.
  response = together.Complete.create(
/usr/local/lib/python3.10/dist-packages/together/legacy/complete.py:23: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment variable instead.
  warnings.warn(API_KEY_WARNING)


⚖️ Legal Text Found:
1. The Indian Penal Code (IPC) Section 80: This section outlines that an act is not considered an offence when it is done by accident or misfortune, without any criminal intention or knowledge, in the doing of a lawful act in a lawful manner by lawful means and with proper care and caution.

    2. Circumstances of the Accident: The key factors that would determine whether your action counts as accidental or not are whether you acted with criminal intention or knowledge, if the act was lawful, done in a lawful manner, by lawful means, and with proper care and caution.

    3. Establishing Absence of Criminal Intention: Make sure you can demonstrate that you did not intentionally cause the accident.

    4. Lawful Act: The action you took should be one that is authorized by law.

    5. Proper Care and Caution: Before undertaking the action, ensure you exercised the caution that a reasonably prudent person would exercise in similar circumstances to prevent an accide

## location awareness

In [12]:
import requests
import json
import re

api_key = "AlzaSymw9AJ0q15jTp12FlRKCqCHrNHH54hWoxJ"
query = "law firms in dharwad"


url = f"https://maps.gomaps.pro/maps/api/place/textsearch/json?query={query}&key={api_key}"
response = requests.get(url).json()


places = []
def get_photo_href(photo_info):
    if photo_info:
        attributions = photo_info.get("html_attributions", [])
        if attributions:
            links = [re.search(r'href="([^"]+)"', attr).group(1) for attr in attributions if 'href="' in attr]
            return links[0] if links else "No attribution link"
    return "No attribution link"


for place in response.get("results", []):
    name = place.get("name", "Unknown")
    address = place.get("formatted_address", "No address found")
    rating = place.get("rating", "No rating")

    photo_info = place.get("photos", [{}])[0]
    reference = get_photo_href(photo_info)
    
    places.append(f"{name}, Address: {address}, Rating: {rating}, reference:{reference}")


places_text = "\n\n".join(places)


prompt = f"""
Based on the given legal places, provide the reference, name, address ,rating  of the most relevant ones to the user:

{places_text}

Only return the 3 most relevant results in a user-friendly way.
"""
response = together.Complete.create(
        model="mistralai/Mistral-7B-Instruct-v0.2",
        prompt=prompt,
        max_tokens=500,
        temperature=0
    )
    

if "choices" in response and response["choices"]:
        
        print(response["choices"][0]["text"].strip())
else:
        print( "Error: No response received from Together AI.")




<ipython-input-12-af620b77f971>:44: DeprecationWarning: Call to deprecated function create.
  response = together.Complete.create(


1. Name: The Law Offices of Omar Zambrano
   Address: 614 W 5th St Ste 200, Los Angeles, CA 90071
   Rating: 4.9 (out of 5)


2. Name: The Law Offices of Joseph H. Low IV
   Address: 11601 Wilshire Blvd #1600, Los Angeles, CA 90025
   Rating: 4.8 (out of 5)


3. Name: The Law Offices of Scott R. Antenstein
   Address: 1900 Avenue of the Stars #2600, Los Angeles, CA 90067
   Rating: 4.7 (out of 5)


## legal advice from lawyers

In [13]:
import json
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings

# Load your dataset
with open("/kaggle/input/legal-advice/answers_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)


# Storage containers
processed_data = []


for entry in data:
    full_text = entry["full_text"]
    answers = entry["answers"]
    question_url = entry["question_url"]

    # Join all answers into one string
    joined_answers = "\n".join(answers)

    # Store only necessary fields
    processed_data.append({
        "question_url": question_url,
        "full_text": full_text,
        "joined_answers": joined_answers
    })

# Save cleaned and processed data
with open("processed_lawyer_data.json", "w", encoding="utf-8") as f:
    json.dump(processed_data, f, indent=2, ensure_ascii=False)


In [14]:
import os
os.environ["WANDB_DISABLED"] = "true"

## summarize question and answer before embedding

In [15]:
import json

# Load the JSON data
with open('/kaggle/working/processed_lawyer_data.json', 'r') as file:
    data = json.load(file)



In [16]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [17]:
model.save_pretrained("bart_summarizer")
tokenizer.save_pretrained("bart_summarizer")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('bart_summarizer/tokenizer_config.json',
 'bart_summarizer/special_tokens_map.json',
 'bart_summarizer/vocab.json',
 'bart_summarizer/merges.txt',
 'bart_summarizer/added_tokens.json')

In [18]:
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration
from tqdm import tqdm
import torch

# Load your dataset
df = pd.read_json("/kaggle/working/processed_lawyer_data.json")  # Or use pd.read_csv(...) depending on your format

# Load BART model and tokenizer
model = BartForConditionalGeneration.from_pretrained("/kaggle/working/bart_summarizer", forced_bos_token_id=0)
tokenizer = BartTokenizer.from_pretrained("/kaggle/working/bart_summarizer")
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

# Function to summarize a single text
def summarize(text, max_input_length=1024, max_output_length=350):
    prompt = (
    "You are a legal assistant. Summarize the following legal situation by extracting:\n"
    "- The legal dispute and its current status\n"
    "- Actions taken by the people involved\n"
    "- The legal question or help being asked\n\n"
    "Text:\n" + text
    )
    inputs = tokenizer(prompt, max_length=1024, truncation=True, return_tensors="pt").to(model.device)

    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=6,
        max_length=max_output_length,
        min_length=120,
        no_repeat_ngram_size=3,
        length_penalty=1.2,
        early_stopping=True
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
# Prepare new columns for summaries
df["fulltext_summary"] = ""
df["answers_summary"] = ""

# Summarize in batches
for idx in tqdm(range(len(df))):
    try:
        full = df.loc[idx, "full_text"]
        ans = df.loc[idx, "joined_answers"]

        df.at[idx, "fulltext_summary"] = summarize(full)
        df.at[idx, "answers_summary"] = summarize(ans)
    except Exception as e:
        print(f"Error at index {idx}: {e}")
        continue

# Save the summarized dataset
df.to_json("summarized_legal_data.json", orient="records", indent=2)


/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
  0%|          | 0/500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1527: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
100%|██████████| 500/500 [37:01<00:00,  4.44s/it]


In [19]:
df = pd.read_json("/kaggle/working/summarized_legal_data.json")

In [20]:
final_legal_advice_data=df[["answers_summary", "fulltext_summary", "question_url"]]

In [21]:
final_legal_advice_data.to_json("final_summarized_data.json", orient="records", indent=2)

## fine tuning inlegalbert

In [22]:
import json

with open("/kaggle/working/final_summarized_data.json", "r") as f:
    data = json.load(f)


In [23]:
from sentence_transformers import SentenceTransformer, models

# Load INLegalBERT model (from HuggingFace or local path)
word_embedding_model = models.Transformer("law-ai/InLegalBERT", max_seq_length=512)

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


In [24]:
from sentence_transformers import losses, InputExample, SentenceTransformer
import torch
from torch.utils.data import Dataset, DataLoader


train_examples = [
    InputExample(texts=[item["fulltext_summary"], item["answers_summary"]], label=1) for item in        data
]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=4)

train_loss = losses.CosineSimilarityLoss(model=model)

# Step 6: Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    show_progress_bar=True
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


In [25]:
model.save("fine-tuned-inlegalbert")


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

## creating faiss index and cross encoder for similarity search

In [26]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("/kaggle/working/fine-tuned-inlegalbert")


In [27]:
import faiss
full_texts = [item["fulltext_summary"] for item in data]
answers = [item["answers_summary"] for item in data]  # in case you want to use later

# Embed all full_texts
full_text_embeddings = model.encode(full_texts, convert_to_numpy=True, show_progress_bar=True)

# Create FAISS index (for cosine similarity use IndexFlatIP and normalize)
dimension = full_text_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)

# Normalize embeddings (important for cosine similarity)
faiss.normalize_L2(full_text_embeddings)

# Add to FAISS
index.add(full_text_embeddings)

# Save the index and texts (optional)
faiss.write_index(index, "final_faiss_fulltext.index")
with open("full_texts.json", "w") as f:
    json.dump(full_texts, f)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [28]:
from sentence_transformers import CrossEncoder

# Load a cross-encoder model trained for relevance ranking
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## legal advice

In [29]:
import numpy as np
import faiss
index=faiss.read_index("/kaggle/working/final_faiss_fulltext.index")
def get_similar_advice(user_question, top_k=10, threshold=0.2):
    # Step 1: Encode query
    query_embedding = model.encode(user_question, convert_to_numpy=True)
    query_embedding = np.array(query_embedding).reshape(1, -1).astype('float32')
    faiss.normalize_L2(query_embedding)

   
    D, I = index.search(query_embedding, top_k)
    similarity_scores = D[0]

    candidates = []
    for idx, i in enumerate(I[0]):
        if i < len(data):
            item = data[i].copy()
            item["similarity"] = float(similarity_scores[idx])
            candidates.append(item)

    # Step 3: Rerank
    rerank_pairs = [(user_question, item["fulltext_summary"]) for item in candidates]
    scores = reranker.predict(rerank_pairs)

    for i, item in enumerate(candidates):
        item["rerank_score"] = scores[i]

    # Step 4: Sort by rerank score
    sorted_candidates = sorted(candidates, key=lambda x: x["rerank_score"], reverse=True)

    
    final_results = []
    for item in sorted_candidates:
        if item["similarity"] >= threshold:
            final_results.append({
                "answers": item["answers_summary"],
                "url": item.get("question_url", None),
                "similarity": round(item["similarity"], 3),
                "rerank_score": round(item["rerank_score"], 3)
            })

    if not final_results:
        return [{
            "answers": "No relevant legal advice found for your query.",
            "url": None,
            "similarity": 0,
            "rerank_score": 0
        }]

    return final_results


In [30]:
user_question = "can i get forced to get divorce because of normal fights"
responses = get_similar_advice(user_question)

for i, res in enumerate(responses, 1):
    print(f"\n--- Response {i} ---")
    print("Answers:\n", res["answers"])
    print("URL:", res["url"])
    print("Similarity:", res["similarity"])
    print("Rerank Score:", res["rerank_score"])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- Response 1 ---
Answers:
 Divorce is not a platform ticket, husband can go to Court and buy it across booking window. Law is in favor of woman. If he files divorce you can file number of cases, he will spend lot of money and time and lose peace of mind. Just ignore their conduct and tell them bluntly to proceed with their plans, you will take counter measure and make them pay for it. It is not that on 366 day husband file a divorce and he will get it. Only in cases of proved adultery, that too not just some incidents of adultery. For living in adultery a husband gets divorce that too after fighting for years in court.
URL: https://www.kaanoon.com/486190/can-husband-force-me-to-give-him-divorce-because-of-normal-fights
Similarity: 0.972
Rerank Score: 7.029

--- Response 2 ---
Answers:
 There's no obligation on you to take care or maintain your in laws or their extended family members. You can stop their entry into your home forcibly, if they become physical you may lodge a criminal 